# 1. Install packages

# This is an inference only notebook part for the mmseg notebook shared by @carnozhao. Since internet cannot be used , and multiple packages need to be installed here , I have added all the packages needed offline, and also cloned Carno's mmseg git repo which is needed in the kernel. There is still some bug here , since all the predictions are appearing empty , and the inference code is copied as it is from Carno's notebook. Either the model training has a bug , or the inference code has a bug. will update asap!

# *EDIT - The inference code is working fine , so is this notebook. There was some issue in the model training , leading to empty predictions.
#   Carno suggested using a larger batch size than 8 , and it worked well for me.*

In [ ]:
import sys
sys.path.append('../input/timm-pytorch-image-models/pytorch-image-models-master')
sys.path.append("../input/segmentation-models-pytorch/segmentation_models.pytorch-0.2.1")
sys.path.append("../input/pretrainedmodels/pretrainedmodels-0.7.4")
sys.path.append("../input/efficientnet-pytorch/EfficientNet-PyTorch-master")

!pip install ../input/mmdetection/addict-2.4.0-py3-none-any.whl > /dev/null
!pip install ../input/mmdetection/yapf-0.31.0-py2.py3-none-any.whl > /dev/null
!pip install ../input/mmdetection/terminaltables-3.1.0-py3-none-any.whl > /dev/null
!pip install ../input/mmdetection/einops* > /dev/null
!pip install ../input/mmdetection/mmcv_full-1.3.17-cp37-cp37m-linux_x86_64.whl > /dev/null

# 2. Install mmsegmentation 

This is from my own [mmseg github repo](https://github.com/CarnoZhao/Kaggle-UWMGIT) (leave a star if you like it!)

I have integrated `segmentation_models_pytorch` in this version of `mmsegmentation`. Although `segmentation_models_pytorch`'s simple Unet performs better than some models of `mmsegmentation`, anyway, `mmsegmentation` is still a good library for segmentation task when you want to compare various models in a unified training pipeline.

I only hard-coded `smp.Unet` in `./mmseg/models/segmentors/smp_models.py`. You can add more `smp` models in it!

In [ ]:
!cp -r ../input/uwmgitt-mmseg/Kaggle-UWMGIT-kaggle_tractseg ./ && cd Kaggle-UWMGIT-kaggle_tractseg  && pip install -e .


# 3. Prepare data

In [ ]:
import os
import glob
import numpy as np
import pandas as pd

import cv2
from PIL import Image
from tqdm.auto import tqdm

## 3.1 Read csv and extract meta info

# 5. Inferencing

## 5.1 Load trained models

In [ ]:
sys.path.append('./Kaggle-UWMGIT-kaggle_tractseg')

from mmseg.apis import init_segmentor, inference_segmentor
from mmcv.utils import config

cfgs = [
    "../input/mmdet-20k-iter-test/config.py",
]

ckpts = [
    "../input/mmdet-20k-iter-test/iter_20000.pth",
]

models = []
for cfg, ckpt in zip(cfgs, ckpts):
    cfg = config.Config.fromfile(cfg)
    cfg.model.backbone.pretrained = None
    cfg.model.test_cfg.logits = True
    cfg.data.test.pipeline[1].transforms.insert(2, dict(type="Normalize", mean=[0,0,0], std=[1,1,1], to_rgb=False))

    model = init_segmentor(cfg, ckpt)
    models.append(model)

## 5.2 Make test submission csv

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import glob
from tqdm.auto import tqdm
from scipy.ndimage import binary_closing, binary_opening, measurements

def rle_encode(img):
    pixels = img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

classes = ['large_bowel', 'small_bowel', 'stomach']
data_dir = "../input/uw-madison-gi-tract-image-segmentation/"
test_dir = os.path.join(data_dir, "test")
sub = pd.read_csv(os.path.join(data_dir, "sample_submission.csv"))
test_images = glob.glob(os.path.join(test_dir, "**", "*.png"), recursive = True)

if len(test_images) == 0:
    test_dir = os.path.join(data_dir, "train")
    sub = pd.read_csv(os.path.join(data_dir, "train.csv"))[["id", "class"]].iloc[:100 * 3]
    sub["predicted"] = ""
    test_images = glob.glob(os.path.join(test_dir, "**", "*.png"), recursive = True)
    
id2img = {_.rsplit("/", 4)[2] + "_" + "_".join(_.rsplit("/", 4)[4].split("_")[:2]): _ for _ in test_images}
sub["file_name"] = sub.id.map(id2img)
sub["days"] = sub.id.apply(lambda x: "_".join(x.split("_")[:2]))
fname2index = {f + c: i for f, c, i in zip(sub.file_name, sub["class"], sub.index)}
sub

## 5.3 Start Inferencing

In [ ]:
subs = []
for day, group in tqdm(sub.groupby("days")):
    imgs = []
    for file_name in group.file_name.unique():
        img = cv2.imread(file_name, cv2.IMREAD_ANYDEPTH)
        old_size = img.shape[:2]
        s = int(os.path.basename(file_name).split("_")[1])
        file_names = [file_name.replace(f"slice_{s:04d}", f"slice_{s + i:04d}") for i in range(-2, 3)]
        file_names = [_ for _ in file_names if os.path.exists(_)]
        imgs = [cv2.imread(file_names[0], cv2.IMREAD_ANYDEPTH)] + [img] + [cv2.imread(file_names[-1], cv2.IMREAD_ANYDEPTH)]
        
        new_img = np.stack(imgs, -1)
        new_img = new_img.astype(np.float32) / new_img.max()

        res = [inference_segmentor(model, new_img)[0] for model in models]
        res = (sum(res) / len(res)).round().astype(np.uint8)
        print(res)
        res = cv2.resize(res, old_size[::-1], interpolation = cv2.INTER_NEAREST)
        for j in range(3):
            rle = rle_encode(res[...,j])
            index = fname2index[file_name + classes[j]]
            sub.loc[index, "predicted"] = rle

## 5.4 Format submission

In [ ]:
sub = sub[["id", "class", "predicted"]]
sub.to_csv("submission.csv", index = False)
sub

Removing the remaining files in the output dir since submission errors out if there are other files than submission.csv in the output.

In [ ]:
!rm -r ./Kaggle-UWMGIT-kaggle_tractseg